In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import joblib
import os

# 1. Cargar datos
df = pd.read_csv('../data/processed/dataset_clasificacion.csv')
X = df[['danceability', 'energy', 'valence', 'tempo']]
y = df['popularity_class']

In [2]:
# 2. Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [3]:
# 3. SMOTE para balancear
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [4]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

grid_rf = GridSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state=42),
    param_grid,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
grid_rf.fit(X_train_res, y_train_res)

print("Mejores parámetros RF:", grid_rf.best_params_)
y_pred_rf = grid_rf.predict(X_test)
print("RF Report:\n", classification_report(y_test, y_pred_rf))


Fitting 3 folds for each of 48 candidates, totalling 144 fits


KeyboardInterrupt: 